# Prueba de API de Dataestur con distintos endpoints

Para documentación, ver [https://www.dataestur.es/apidata/](https://www.dataestur.es/apidata/)

Las respuestas de la API de dataestur no son archivos json, son excel, por lo cual se debe estructurar el codigo de forma adecuada para leer esta respuesta. IMPORTANTE: Se requiere BytesIO para leer archivos .xlsx con pandas, ya que este no cuenta con su propio motor integrado inmediatamente.

In [1]:
import requests

url = "https://dataestur.azure-api.net/API-SEGITTUR-v1/ETR_DL"
params = {
    "desde": 2024
}

response = requests.get(url, params=params)

print("Status code:", response.status_code)
print("Content-Type:", response.headers.get("Content-Type"))
print("Raw text response (first 500 chars):")
print(response.text[:500])  # muestra una parte de la respuesta

# Intenta parsear como JSON de forma segura
try:
    data = response.json()
    print("JSON loaded successfully.")
except ValueError:
    print("La respuesta no es un JSON válido.")


Status code: 200
Content-Type: application/vnd.ms-excel
Raw text response (first 500 chars):
PK   V��ZAMb�   �      docProps/app.xmlM�=1D��q��A�Bb@�R��{/�dC�B~�9��noF�
�rmީ�5�&����;i^PK   V��Z��ɭ�   �     docProps/core.xml���N�0�_e�u�iEY.�����$�(�jM%F�ޞ�ln���g[���|��@5����6
��ّ��h��t,Ѧp��)=��6'}@(��!i�I�(��l�.Jkf��$��A�-E��K\��aJfr��L�}_�ˉKqx{ھL��uI�3%�&��.��"	ߊ�2���v�:{\g��u���d�d�*gU^�v��N�������&t����?�W������'PK   V��Z�\�#  �'     xl/theme/theme1.xml�Z[s�8~��
La respuesta no es un JSON válido.


Cuando descargas un archivo Excel con requests, lo que recibes es un blob de bytes (response.content), no un archivo físico.

Entonces usas BytesIO(response.content) para simular un archivo que pandas puede leer como si estuviera en disco, sin tener que guardarlo primero.

In [2]:
import requests
import pandas as pd
from io import BytesIO

url = "https://dataestur.azure-api.net/API-SEGITTUR-v1/ETR_DL"
params = {"desde": 2024}

response = requests.get(url, params=params)

if response.status_code == 200 and response.headers['Content-Type'] == 'application/vnd.ms-excel':
    # Leer el contenido Excel desde la memoria
    excel_file = BytesIO(response.content)

    # Leer todas las hojas disponibles
    xls = pd.ExcelFile(excel_file)
    print("Hojas disponibles:", xls.sheet_names)

    # Leer la hoja principal (usualmente la primera)
    df = pd.read_excel(xls, sheet_name=0)
    print(df.head())
else:
    print("Error: no se recibió un archivo Excel válido.")


Hojas disponibles: ['Sheet1']
    AÑO  MES  PERNOCTACIONES   GASTO_TOTAL    VIAJES
0  2015    2        29977727  1.675889e+09  10134462
1  2015    3        33645821  2.027952e+09  11608193
2  2015    4        58668750  3.246488e+09  16336628
3  2015    5        43416018  2.697760e+09  14814300
4  2015    6        48044157  3.041589e+09  15605072


In [3]:
df

,AÑO,MES,PERNOCTACIONES,GASTO_TOTAL,VIAJES
0,2015,2,29977727,1.675889e+09,10134462
1,2015,3,33645821,2.027952e+09,11608193
2,2015,4,58668750,3.246488e+09,16336628
3,2015,5,43416018,2.697760e+09,14814300
4,2015,6,48044157,3.041589e+09,15605072
...,...,...,...,...,...
114,2024,8,197265468,1.118094e+10,24867614
115,2024,9,99602140,6.181098e+09,16101193
116,2024,10,41155800,4.077141e+09,12486330
117,2024,11,38100784,3.716773e+09,11561861


Al parecer los parametros se leen de forma distinta, como se puede ver en la URL que retornan los test de la misma pagina

In [4]:
import requests
import pandas as pd
from io import BytesIO

url = "https://dataestur.azure-api.net/API-SEGITTUR-v1/ETR_DL?desde%20%28a%C3%B1o%29=2022&hasta%20%28a%C3%B1o%29=2023&Pa%C3%ADs%20destino=Espa%C3%B1a&CCAA%20de%20residencia=Todos&CCAA%20de%20destino=Todos"
params = {"accept": "application/vnd.ms-excel"}

response = requests.get(url, params=params)

if response.status_code == 200 and response.headers['Content-Type'] == 'application/vnd.ms-excel':
    # Leer el contenido Excel desde la memoria
    excel_file = BytesIO(response.content)

    # Leer todas las hojas disponibles
    xls = pd.ExcelFile(excel_file)
    print("Hojas disponibles:", xls.sheet_names)

    # Leer la hoja principal (usualmente la primera)
    df = pd.read_excel(xls, sheet_name=0)
    print(df.head())
else:
    print("Error: no se recibió un archivo Excel válido.")


Hojas disponibles: ['Sheet1']
    AÑO  MES PAIS_DESTINO  PERNOCTACIONES   GASTO_TOTAL    VIAJES
0  2022    1       España        33317294  1.700136e+09   9825649
1  2022    2       España        24994529  1.693647e+09  10183578
2  2022    3       España        25677123  1.782100e+09   9771377
3  2022    4       España        48830922  3.047026e+09  14464860
4  2022    5       España        36189836  2.618699e+09  13639382


In [5]:
df

,AÑO,MES,PAIS_DESTINO,PERNOCTACIONES,GASTO_TOTAL,VIAJES
0,2022,1,España,33317294,1.700136e+09,9825649
1,2022,2,España,24994529,1.693647e+09,10183578
2,2022,3,España,25677123,1.782100e+09,9771377
3,2022,4,España,48830922,3.047026e+09,14464860
4,2022,5,España,36189836,2.618699e+09,13639382
5,2022,6,España,38203431,2.864968e+09,12890375
6,2022,7,España,76706679,4.601212e+09,17492327
7,2022,8,España,150193145,6.818803e+09,20499769
8,2022,9,España,73972871,3.431108e+09,12664666
9,2022,10,España,37179908,2.557838e+09,12141651


In [9]:
import requests
import pandas as pd
from io import BytesIO

url = "https://dataestur.azure-api.net/API-SEGITTUR-v1/ETR_DL"

params = {
    "desde (año)": 2022,
    "hasta (año)": 2024,
    "País destino": "España",
    "CCAA de residencia": "Todos",
    "CCAA de destino": "Todos"
}

response = requests.get(url, params=params)

if response.status_code == 200 and response.headers['Content-Type'] == 'application/vnd.ms-excel':
    excel_file = BytesIO(response.content)
    xls = pd.ExcelFile(excel_file)
    print("Hojas disponibles:", xls.sheet_names)

    df = pd.read_excel(xls, sheet_name=0)
    print(df.head())
else:
    print("Error: no se recibió un archivo Excel válido.")


Hojas disponibles: ['Sheet1']
    AÑO  MES PAIS_DESTINO  PERNOCTACIONES   GASTO_TOTAL    VIAJES
0  2022    1       España        33317294  1.700136e+09   9825649
1  2022    2       España        24994529  1.693647e+09  10183578
2  2022    3       España        25677123  1.782100e+09   9771377
3  2022    4       España        48830922  3.047026e+09  14464860
4  2022    5       España        36189836  2.618699e+09  13639382


In [10]:
df

,AÑO,MES,PAIS_DESTINO,PERNOCTACIONES,GASTO_TOTAL,VIAJES
0,2022,1,España,33317294,1.700136e+09,9825649
1,2022,2,España,24994529,1.693647e+09,10183578
2,2022,3,España,25677123,1.782100e+09,9771377
3,2022,4,España,48830922,3.047026e+09,14464860
4,2022,5,España,36189836,2.618699e+09,13639382
5,2022,6,España,38203431,2.864968e+09,12890375
6,2022,7,España,76706679,4.601212e+09,17492327
7,2022,8,España,150193145,6.818803e+09,20499769
8,2022,9,España,73972871,3.431108e+09,12664666
9,2022,10,España,37179908,2.557838e+09,12141651


In [1]:
import requests
import pandas as pd
from io import BytesIO

# URL base del endpoint
url = "https://dataestur.azure-api.net/API-SEGITTUR-v1/FRONTUR_DL"

# Parámetros de la consulta
params = {
    "desde (año)": 2020,
    "hasta (año)": 2024,
    "País de residencia": "Todos",
    "Tipo de visitante": "Todos",
    "CCAA de destino": "Todos"
}

# Encabezados (para asegurar que aceptamos el formato Excel)
headers = {
    "accept": "application/vnd.ms-excel"
}

# Realizar la solicitud GET
response = requests.get(url, params=params, headers=headers)

# Verificar la respuesta
if response.status_code == 200 and response.headers['Content-Type'] == 'application/vnd.ms-excel':
    # Leer el archivo Excel desde la respuesta
    excel_file = BytesIO(response.content)
    
    # Leer las hojas disponibles en el archivo
    xls = pd.ExcelFile(excel_file)
    print("Hojas disponibles:", xls.sheet_names)

    # Leer la primera hoja (si no estás seguro de cuál es la principal)
    df = pd.read_excel(xls, sheet_name=0)
    
    # Ver las primeras filas del DataFrame
    print(df.head())

else:
    print(f"Error: No se recibió un archivo Excel válido (status code {response.status_code})")


Hojas disponibles: ['Sheet1']
    AÑO  MES PAIS_RESIDENCIA TIPO_VISITANTE       CCAA_DESTINO  VISITANTES
0  2020    1        Alemania  Excursionista          Andalucía        3446
1  2020    1        Alemania  Excursionista          Baleares         6103
2  2020    1        Alemania  Excursionista          Canarias        23521
3  2020    1        Alemania  Excursionista  Castilla y León           567
4  2020    1        Alemania  Excursionista         Cataluña          7139


In [2]:
df

,AÑO,MES,PAIS_RESIDENCIA,TIPO_VISITANTE,CCAA_DESTINO,VISITANTES
0,2020,1,Alemania,Excursionista,Andalucía,3446
1,2020,1,Alemania,Excursionista,Baleares,6103
2,2020,1,Alemania,Excursionista,Canarias,23521
3,2020,1,Alemania,Excursionista,Castilla y León,567
4,2020,1,Alemania,Excursionista,Cataluña,7139
...,...,...,...,...,...,...
19806,2024,12,Suiza,Turista,Com.Valenciana,16370
19807,2024,12,Suiza,Turista,Galicia,2588
19808,2024,12,Suiza,Turista,País Vasco,1610
19809,2024,12,Suiza,Turista,Principado de Asturias,28


In [9]:
import requests
import pandas as pd
from io import BytesIO

# URL base del endpoint
url = "https://dataestur.azure-api.net/API-SEGITTUR-v1/ACTIVIDADES_OCIO_DL"

# Parámetros de la consulta. Sin parametros se obtienen todos los datos disponibles
params = {
    ##"desde (año)": 2023,
    ##"hasta (año)": 2024 ##,
    ##"País de residencia": "Todos",
    ##"Tipo de visitante": "Todos",
    ##"CCAA de destino": "Todos"
}

# Encabezados (para asegurar que aceptamos el formato Excel)
headers = {
    "accept": "application/vnd.ms-excel"
}

# Realizar la solicitud GET
response = requests.get(url, params=params, headers=headers)

# Verificar la respuesta
if response.status_code == 200 and response.headers['Content-Type'] == 'application/vnd.ms-excel':
    # Leer el archivo Excel desde la respuesta
    excel_file = BytesIO(response.content)
    
    # Leer las hojas disponibles en el archivo
    xls = pd.ExcelFile(excel_file)
    print("Hojas disponibles:", xls.sheet_names)

    # Leer la primera hoja (si no estás seguro de cuál es la principal)
    df = pd.read_excel(xls, sheet_name=0)
    
    # Ver las primeras filas del DataFrame
    print(df.head())

else:
    print(f"Error: No se recibió un archivo Excel válido (status code {response.status_code})")


Hojas disponibles: ['Sheet1']
    AÑO  MES      PRODUCTO                CATEGORIA        CCAA    PROVINCIA  \
0  2023    7  Espectaculos  Cultura, teatro y danza  País Vasco  Araba/Álava   
1  2023    7  Espectaculos  Cultura, teatro y danza  País Vasco  Araba/Álava   
2  2023    7  Espectaculos                   Música  País Vasco  Araba/Álava   
3  2023    7   Actividades   Actividades infantiles  País Vasco  Araba/Álava   
4  2023    7  Espectaculos  Cultura, teatro y danza  País Vasco  Araba/Álava   

   ENTRADAS  VISITAS_PAGINAS  GASTO_TOTAL  PRECIO_MEDIO_ENTRADA  TRANSACCIONES  
0        35              135       420.00                 12.00             12  
1         2                8         6.46                  3.23              1  
2       310             4171      3357.30                 10.83            153  
3        39              151       207.87                  5.33             17  
4       290             1123      2726.00                  9.40            124  


In [8]:
df

,AÑO,MES,PRODUCTO,CATEGORIA,CCAA,PROVINCIA,ENTRADAS,VISITAS_PAGINAS,GASTO_TOTAL,PRECIO_MEDIO_ENTRADA,TRANSACCIONES
0,2023,7,Espectaculos,"Cultura, teatro y danza",País Vasco,Araba/Álava,35,135,420.0000,12.0000,12
1,2023,7,Espectaculos,"Cultura, teatro y danza",País Vasco,Araba/Álava,2,8,6.4600,3.2300,1
2,2023,7,Espectaculos,Música,País Vasco,Araba/Álava,310,4171,3357.3000,10.8300,153
3,2023,7,Actividades,Actividades infantiles,País Vasco,Araba/Álava,39,151,207.8700,5.3300,17
4,2023,7,Espectaculos,"Cultura, teatro y danza",País Vasco,Araba/Álava,290,1123,2726.0000,9.4000,124
...,...,...,...,...,...,...,...,...,...,...,...
12575,2024,12,Actividades,Deportes y aventuras,Total Nacional,Total Nacional,6572,30716,94688.7188,14.4079,2983
12576,2024,12,Actividades,Actividades infantiles,Total Nacional,Total Nacional,46116,213508,576122.5764,12.4929,13890
12577,2024,12,Espectaculos,"Cultura, teatro y danza",Total Nacional,Total Nacional,58427,266703,710080.8591,12.1533,21044
12578,2024,12,Espectaculos,Museo y exposiciones,Total Nacional,Total Nacional,5163,24410,59219.6100,11.4700,1310
